In [1]:
import pandas as pd
import matplotlib.pyplot as plt

### PART I: Analyzing Netflix quotes (close price) for 2019

#### Read only __date__ and __close price__ from the downloaded quotes file of Netflix from NASDAQ
___

In [2]:
quotes = pd.read_csv('../data/nasdaq_netlflix_historical_quotes.csv')
quotes = quotes.rename(columns={'Date': 'date', ' Close/Last': 'close_price'})
quotes = quotes[['date', 'close_price']]

#### Cleaning __date__
___
- Changing data type
- Since I will be looking at 2019's data so keeping only the relevant dates

In [3]:
quotes['date'] = quotes['date'].astype('datetime64[ns]')
quotes = quotes[(quotes.date >= '2019-01-01') & (quotes.date <= '2019-12-31')]

#### Cleaning __close price__
___
- Removing '$' sign
- Changing data type

In [4]:
quotes['close_price'] = quotes['close_price'].apply(lambda r: r.replace(" $", ""))
quotes['close_price'] = quotes['close_price'].astype('double')

#### Extracting more information
___
- The close price in itself is not telling me much, so I need something else.
- I am actually interested in knowing whether price increased/decreased and so that's what I'll compute in __diff__ column.

In [5]:
quotes['diff'] = quotes.close_price.diff()

#### Understanding the data
___
- Since price fluctuations are floating point integers I first bucket them in bins of size=1, i.e., all price jumps between 0.0 to 1.0 will fall in the category bucket (0, 1]
- To make working with buckets easier (since it is of interval type) I add the __code__ column having category codes.
- I then plot a histogram to visualize the data and see how the price fluctuations are distributed.

In [ ]:
quotes['bucket'] = pd.cut(quotes['diff'], list(range(round(quotes['diff'].min())-1, round(quotes['diff'].max())+1)))

In [ ]:
quotes['code'] = quotes.bucket.cat.codes

In [ ]:
plt.figure(figsize=(8,5))
plt.xlabel("Code Bins") 
plt.ylabel("Frequency") 
plt.hist(quotes['code'], color="purple", edgecolor="white")

##### The above graph shows that it is a normal distribution (which makes sense since in general price variations will fall within a range and will not vary drastically all year round).
##### Majority of the values fall in buckets with codes around 25.
##### Since I am concerned about outlier cases when extraordinary variation was observed, I will focus on the buckets outside this range.

#### Analyzing the outliers (when more than normal variation in price was observed)
___

In [ ]:
outliers = quotes[(quotes.code < 5) | (quotes.code > 40)][['date', 'diff']]
plt.figure(figsize=(15,5))
plt.xlabel("Date") 
plt.ylabel("Close Price Variation") 
plt.bar(outliers['date'], outliers['diff'], color ='orange', width = 10, edgecolor='maroon', linewidth=2)

In [ ]:
outliers = outliers.set_index(('date'))
outliers

##### From the observations above we can see that the price fluctuated maximum around mid-July.

### PART II: Analyzing Google trends data for Stranger Things (a popular Netflix show) in 2019

#### Reading the file downloaded from Google Trends website
___

In [ ]:
trends = pd.read_csv('../data/stanger_things_google_trends_2019.csv', skiprows=1, sep=",")

#### Plotting the trends data
___

In [ ]:
plt.figure(figsize=(15,5))
plt.xticks(rotation=90)
plt.xlabel("Week") 
plt.ylabel("Relative Search Interest") 
plt.plot(trends.set_index('Week'), "gs-", linewidth=2, markersize=8)

##### If we look at the plot above then the peak occured around first week of July.
##### This happens to be the around the release date (4th July) of the third season of Stranger Things.
##### Could it be that this somehow impacted the price variation we saw above in Neflix stocks around this time?
##### Maybe Yes, Maybe No.